In [ ]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import time
import re
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')

In [ ]:
#mentorship.csv contains mentorship relationships between researchers and has 8 variables: relationship ID (CID), mentee’s person ID, mentor’s person ID, mentorship type, the institution where the
#mentorship happened, institution MAG ID, and the start year and stop year of the interaction.
mentorship=pd.read_csv('/content/drive/MyDrive/academic_tree/data/mentorship.csv')

In [ ]:
#relationship ID (CID),
mentorship
#stop year 最全611925 /743176

CID  MenteeID  MentorID  MentorshipType  \
0             2         2         3               1   
1             3         4         3               2   
2             5         6         3               1   
3             6     18761         9               1   
4             7        10        16               2   
...         ...       ...       ...             ...   
743171  1784798    839679    287445               1   
743172  1784800    796638    839680               1   
743173  1784802    839681    699264               1   
743174  1784803    699264     38114               1   
743175  1784805    747127    155716               1   

                                              Institution  InstitutionMAGID  \
0                      University of California, Berkeley        95457486.0   
1                      University of California, Berkeley        95457486.0   
2                      University of California, Berkeley        95457486.0   
3                      University of California, Berkeley        95457486.0   
4                      Washington University, Saint Louis       204465549.0   
...                                                   ...               ...   
743171               University of Massachusetts, Amherst        24603500.0   
743172                                University of Wales        97429440.0   
743173            University of California, Santa Barbara       154570441.0   
743174                                                NaN               NaN   
743175  Northeast Ohio Medical University; Kent State ...               NaN   

        StartYear  StopYear  
0            2000      2005  
1            2003      2006  
2            2002      2008  
3              -1      1984  
4              -1        -1  
...           ...       ...  
743171         -1      1991  
743172         -1      1986  
743173       2017        -1  
743174         -1        -1  
743175       2018      2019  

[743176 rows x 8 columns]

In [ ]:
mentorship=mentorship[ (mentorship['MentorshipType'] == 1)|(mentorship['MentorshipType'] == 2)]

In [ ]:
mentorship['StartYear'] = mentorship['StartYear'].replace(-1, np.nan)
mentorship['StopYear'] = mentorship['StopYear'].replace(-1, np.nan)

<ipython-input-5-1ce5aa6adcbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mentorship['StartYear'] = mentorship['StartYear'].replace(-1, np.nan)
<ipython-input-5-1ce5aa6adcbd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mentorship['StopYear'] = mentorship['StopYear'].replace(-1, np.nan)


时间修补

先保证时间登记的是对的再说 修正时间 存在几个问题

(1)StopYear<StartYear

(2)MenteeID_type1['Diff']=MenteeID_type1.StopYear-MenteeID_type1.StartYear

MenteeID_type1=MenteeID_type1[MenteeID_type1['Diff']<=np.percentile(MenteeID_type1['Diff'], 99.5)]



In [ ]:
#1
mask=(mentorship['StartYear'] < 1900)| (mentorship['StopYear'] < 1900)
mentorship.loc[mask&(mentorship['StopYear'] > 1900),'StartYear']=mentorship.loc[mask&(mentorship['StopYear'] > 1900),'StopYear']-4

In [ ]:
#里面因为笔误把1973写成1873但是staring year1971,因此按照逻辑是1973,我们用样本的平均值填补
mask=(mentorship['StartYear'] < 1900)| (mentorship['StopYear'] < 1900)

In [ ]:
mentorship.loc[(mentorship['MentorshipType']==1)&mask&(mentorship['StartYear'] > 1900)]

CID  MenteeID  MentorID  MentorshipType  \
79270     97750     77314      3367               1   
394174   891293     23367    347919               1   
559390  1298572    686327    686328               1   
562833  1305768    688023    688205               1   
713491  1706178    817850    146480               1   

                                              Institution  InstitutionMAGID  \
79270                              Stony Brook University        59553526.0   
394174  Ontario Institute for Studies in Education at ...               NaN   
559390               University of Leeds (United Kingdom)       130828816.0   
562833                     Universidade Estadual Paulista               NaN   
713491                      University of Texas at Austin        86519309.0   

        StartYear  StopYear  
79270      1978.0    1979.0  
394174     1967.0    1970.0  
559390     1971.0    1976.0  
562833     2008.0    2010.0  
713491     2015.0    2019.0

In [ ]:
corrections = {
    1879: 1979,
    1700: 1970,
    1776: 1976,
    1010: 2010,
    1019: 2019
}
condition = (mentorship['MentorshipType'] == 1) & mask & (mentorship['StartYear'] > 1900)
mentorship.loc[condition, 'StopYear'] = mentorship.loc[condition, 'StopYear'].replace(corrections)

In [ ]:
mentorship.loc[(mentorship['MentorshipType']==2)&mask&(mentorship['StartYear'] > 1900)]

CID  MenteeID  MentorID  MentorshipType  \
30606     38327     27147      4692               2   
31294     39190     23896      1136               2   
38889     48694     33243     21794               2   
84146    103683     81287     62681               2   
264056   589597    285241    285240               2   
555797  1290183     19783      2279               2   
649891  1540086     19723    769324               2   
715183  1711450    754103     55613               2   

                                              Institution  InstitutionMAGID  \
30606                                  CONICET, Argentina               NaN   
31294                   University of Washington, Seattle       201448701.0   
38889   Harvard Medical School and Massachusetts Gener...               NaN   
84146             University of California, San Francisco       180670191.0   
264056                                 Harvard University       136199984.0   
555797   The Salk Institute for Biological Studies & UCSD               NaN   
649891                              University of Calgary       168635309.0   
715183                                                NaN               NaN   

        StartYear  StopYear  
30606      1983.0    1895.0  
31294      1971.0    1873.0  
38889      1984.0    1886.0  
84146      2005.0    1012.0  
264056     1977.0    1879.0  
555797     1993.0    1006.0  
649891     1976.0    1078.0  
715183     1988.0    1001.0

In [ ]:
stop_year_corrections = {
    1895: 1985,
    1873: 1973,
    1886: 1986,
    1012: 2012,
    1879: 1979,
    1006: 1996,
    1078: 1978,
    1001: 1991
}

# 筛选需要修正的行（基于CID）
condition = (mentorship['MentorshipType']==2)&mask&(mentorship['StartYear'] > 1900)
# 应用修正
mentorship.loc[condition, 'StopYear'] = mentorship.loc[condition, 'StopYear'].replace(stop_year_corrections)


In [ ]:
#2 StopYear<StartYear
#mentorship_filtered[mentorship_filtered.StopYear<mentorship_filtered.StartYear]
# 条件筛选
condition = (mentorship['StopYear'] < mentorship['StartYear'])
# 交换内容（保持列名不变）
error_records = mentorship.loc[condition]
mentorship.loc[condition, ['StartYear', 'StopYear']] = mentorship.loc[condition, ['StopYear', 'StartYear']].values

In [ ]:
#3
mentorship['Diff']=mentorship.StopYear-mentorship.StartYear

<ipython-input-14-c8baa2f1c8d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mentorship['Diff']=mentorship.StopYear-mentorship.StartYear


In [ ]:
mentorship.sort_values('Diff',ascending=False).head(20)

CID  MenteeID  MentorID  MentorshipType  \
679705  1617892     30640    791939               2   
736533  1767587    834719    834704               1   
695857  1659629    804254     73945               1   
31378     39314     27878     27888               1   
539398  1250200     50937      3119               1   
430364   976318    175900       256               1   
643844  1525507    765074    765185               2   
539389  1250185    662470       149               1   
676044  1608030     13121    789227               1   
20693     25774     18724     18725               1   
521212  1201232     11277     10430               1   
664390  1578047    780554    780553               1   
111121   109711     85634     85637               1   
670586  1594802    785364    541626               1   
7421       8885      6794       558               1   
706895  1688394    812971    583311               1   
626072  1476638    752212    752216               1   
707126  1689038    798763    813190               1   
662475  1572489    778987     16281               2   
707127  1689040    798763    813191               1   

                                              Institution  InstitutionMAGID  \
679705  Schlesische Friedrich-Wilhelms-Universit&auml;...               NaN   
736533                University of California, Riverside       103635307.0   
695857                                 Harvard University       136199984.0   
31378                                  Harvard University       136199984.0   
539398              University of California, Los Angeles       161318765.0   
430364                                 Harvard University       136199984.0   
643844                                                NaN               NaN   
539389                                 Harvard University       136199984.0   
676044                                New York University        57206974.0   
20693                        University of St. Petersburg               NaN   
521212                                                NaN               NaN   
664390                    University of California, Davis        84218800.0   
111121                  University of Washington, Seattle       201448701.0   
670586                  University of Washington, Seattle       201448701.0   
7421      Max Planck Institute for Biological Cybernetics               NaN   
706895                University of California, San Diego        36258959.0   
626072          University of British Columbia, Vancouver       141945490.0   
707126                  University of Michigan, Ann Arbor        27837315.0   
662475            David Geffen School of Medicine at UCLA               NaN   
707127                  University of Michigan, Ann Arbor        27837315.0   

        StartYear  StopYear   Diff  
679705     1913.0    2014.0  101.0  
736533     1910.0    2010.0  100.0  
695857     1904.0    1988.0   84.0  
31378      1933.0    1999.0   66.0  
539398     1931.0    1990.0   59.0  
430364     1931.0    1986.0   55.0  
643844     1955.0    2008.0   53.0  
539389     1923.0    1970.0   47.0  
676044     1975.0    2019.0   44.0  
20693      1849.0    1884.0   35.0  
521212     1933.0    1967.0   34.0  
664390     1963.0    1997.0   34.0  
111121     1980.0    2014.0   34.0  
670586     1978.0    2011.0   33.0  
7421       1966.0    1999.0   33.0  
706895     1966.0    1999.0   33.0  
626072     1988.0    2018.0   30.0  
707126     1923.0    1953.0   30.0  
662475     1987.0    2017.0   30.0  
707127     1923.0    1953.0   30.0

In [ ]:
mask = (mentorship["Diff"] <= np.nanpercentile(mentorship['Diff'], 99)) | (mentorship["Diff"].isna())
mentorship=mentorship[mask]

In [ ]:
mentorship

CID  MenteeID  MentorID  MentorshipType  \
0             2         2         3               1   
1             3         4         3               2   
2             5         6         3               1   
3             6     18761         9               1   
4             7        10        16               2   
...         ...       ...       ...             ...   
743171  1784798    839679    287445               1   
743172  1784800    796638    839680               1   
743173  1784802    839681    699264               1   
743174  1784803    699264     38114               1   
743175  1784805    747127    155716               1   

                                              Institution  InstitutionMAGID  \
0                      University of California, Berkeley        95457486.0   
1                      University of California, Berkeley        95457486.0   
2                      University of California, Berkeley        95457486.0   
3                      University of California, Berkeley        95457486.0   
4                      Washington University, Saint Louis       204465549.0   
...                                                   ...               ...   
743171               University of Massachusetts, Amherst        24603500.0   
743172                                University of Wales        97429440.0   
743173            University of California, Santa Barbara       154570441.0   
743174                                                NaN               NaN   
743175  Northeast Ohio Medical University; Kent State ...               NaN   

        StartYear  StopYear  Diff  
0          2000.0    2005.0   5.0  
1          2003.0    2006.0   3.0  
2          2002.0    2008.0   6.0  
3             NaN    1984.0   NaN  
4             NaN       NaN   NaN  
...           ...       ...   ...  
743171        NaN    1991.0   NaN  
743172        NaN    1986.0   NaN  
743173     2017.0       NaN   NaN  
743174        NaN       NaN   NaN  
743175     2018.0    2019.0   1.0  

[698487 rows x 9 columns]

In [ ]:
empty_records = mentorship[(mentorship['StartYear'].isnull()) & (mentorship['StopYear'].isnull())]
# 按MenteeID分组，并统计每个MenteeID下满足条件的记录数量
empty_count_per_id = empty_records.groupby('MenteeID').size()

In [ ]:
empty_count_per_id

MenteeID
3         3
7         2
10        3
11        1
13        1
         ..
839664    1
839665    1
839673    1
839674    1
839676    1
Length: 69983, dtype: int64

In [ ]:
# 找出满足条件的MenteeID，即满足条件的记录数量等于该MenteeID的总记录数量
result = empty_count_per_id[empty_count_per_id == mentorship.groupby('MenteeID').size()]


开始根据时间段进行时间填充

In [ ]:
resume_df = mentorship.sort_values(['MenteeID', 'StopYear'])

In [ ]:
resume_df.head(10)

CID  MenteeID  MentorID  MentorshipType  \
113068     1         1         3               1   
308      346         1       184               2   
0          2         2         3               1   
1102    1256         2        72               2   
5          8         3        16               2   
6          9         3        26               1   
574      643         3       595               1   
40        48         4        52               1   
1          3         4         3               2   
8015    9624         4       788               2   

                                 Institution  InstitutionMAGID  StartYear  \
113068    University of California, Berkeley        95457486.0     1998.0   
308     University of Maryland, College Park        66946132.0     2005.0   
0         University of California, Berkeley        95457486.0     2000.0   
1102                         Duke University       170897317.0     2005.0   
5         Washington University, Saint Louis       204465549.0        NaN   
6                            Yale University        32971472.0        NaN   
574                          Yale University        32971472.0        NaN   
40                   University of Cambridge          241749.0     1999.0   
1         University of California, Berkeley        95457486.0     2003.0   
8015                    University of Oxford        40120149.0     2006.0   

        StopYear  Diff  
113068    2005.0   7.0  
308       2012.0   7.0  
0         2005.0   5.0  
1102      2011.0   6.0  
5            NaN   NaN  
6            NaN   NaN  
574          NaN   NaN  
40        2002.0   3.0  
1         2006.0   3.0  
8015         NaN   NaN

要看一个mentee id 下面有几段学习经历，有的读了两个PHD和好几段POSTDOC

In [ ]:
mentor_stats = (
    resume_df.groupby('MenteeID')['MentorshipType']
    .agg(['count', lambda x: list(x)])  # 使用list而非unique
    .reset_index()
    .rename(columns={'<lambda_0>': 'mentorship_types'})  # 重命名列
)

mentor_stats['unique_types'] = mentor_stats['mentorship_types'].apply(lambda x: set(x))
# 扩展mentor_stats，分别统计Type 1和Type 2的数量
mentor_stats['count_type1'] = mentor_stats['mentorship_types'].apply(lambda x: x.count(1))
mentor_stats['count_type2'] = mentor_stats['mentorship_types'].apply(lambda x: x.count(2))

In [ ]:
mentor_stats

MenteeID  count mentorship_types unique_types  count_type1  \
0              1      2           [1, 2]       {1, 2}            1   
1              2      2           [1, 2]       {1, 2}            1   
2              3      3        [2, 1, 1]       {1, 2}            2   
3              4      3        [1, 2, 2]       {1, 2}            1   
4              5      2           [1, 2]       {1, 2}            1   
...          ...    ...              ...          ...          ...   
619627    839675      1              [1]          {1}            1   
619628    839676      1              [2]          {2}            0   
619629    839677      1              [1]          {1}            1   
619630    839679      1              [1]          {1}            1   
619631    839681      1              [1]          {1}            1   

        count_type2  
0                 1  
1                 1  
2                 1  
3                 2  
4                 1  
...             ...  
619627            0  
619628            1  
619629            0  
619630            0  
619631            0  

[619632 rows x 6 columns]

In [ ]:
mentorship.MentorshipType.value_counts()

MentorshipType
1    629978
2     68509
Name: count, dtype: int64

In [ ]:
# 定义修正后的统计函数
def count_mentorship_patterns(df):
    patterns = {
        "0次PhD & 1次Postdoc": (0, 1),
        "1次PhD & 0次Postdoc": (1, 0),
        "1次PhD & 1次Postdoc": (1, 1),
        "1次PhD & >1次Postdoc": (1, ">1"),
        "0次PhD & >1次Postdoc": (0, ">1"),
        ">1次PhD & 1次Postdoc": (">1", 1),
        ">1次PhD & >1次Postdoc": (">1", ">1"),
        ">1次PhD & 0次Postdoc": (">1", 0)
    }

    results = {}
    total = 0

    for desc, condition in patterns.items():
        cond1, cond2 = condition

        # 构建条件掩码
        mask1 = (df['count_type1'] > 1) if cond1 == ">1" else (df['count_type1'] == cond1)
        mask2 = (df['count_type2'] > 1) if cond2 == ">1" else (df['count_type2'] == cond2)

        cnt = len(df[mask1 & mask2])
        results[desc] = cnt
        total += cnt

    # 计算未分类记录（如果需要）
    results["未分类记录"] = len(df) - total

    return pd.DataFrame.from_dict(results, orient='index', columns=['Count'])

# 执行统计
result_df = count_mentorship_patterns(mentor_stats)
print(result_df.sort_values('Count', ascending=False))
#​619,632

                      Count
1次PhD & 0次Postdoc    542453
1次PhD & 1次Postdoc     24435
0次PhD & 1次Postdoc     19723
>1次PhD & 0次Postdoc    19606
1次PhD & >1次Postdoc     5827
>1次PhD & 1次Postdoc     4362
>1次PhD & >1次Postdoc    1668
0次PhD & >1次Postdoc     1558
未分类记录                     0


In [ ]:
# 定义类别编码规则
category_map = {
    1: "1次PhD & 0次Postdoc",
    2: "1次PhD & 1次Postdoc",
    3: "1次PhD & >1次Postdoc",
    4: ">1次PhD & 0次Postdoc",
    5: ">1次PhD & 1次Postdoc",
    6: ">1次PhD & >1次Postdoc",
    7: "0次PhD & 1次Postdoc",
    8: "0次PhD & >1次Postdoc"
}

mentor_stats['category'] = 0
mentor_stats.loc[(mentor_stats['count_type1'] == 1) & (mentor_stats['count_type2'] == 0), 'category'] = 1
mentor_stats.loc[(mentor_stats['count_type1'] == 1) & (mentor_stats['count_type2'] == 1), 'category'] = 2
mentor_stats.loc[(mentor_stats['count_type1'] == 1) & (mentor_stats['count_type2'] > 1), 'category'] = 3
mentor_stats.loc[(mentor_stats['count_type1'] > 1) & (mentor_stats['count_type2'] == 0), 'category'] = 4
mentor_stats.loc[(mentor_stats['count_type1'] > 1) & (mentor_stats['count_type2'] == 1), 'category'] = 5
mentor_stats.loc[(mentor_stats['count_type1'] > 1) & (mentor_stats['count_type2'] > 1), 'category'] = 6
mentor_stats.loc[(mentor_stats['count_type1'] == 0) & (mentor_stats['count_type2'] == 1), 'category'] = 7
mentor_stats.loc[(mentor_stats['count_type1'] == 0) & (mentor_stats['count_type2'] > 1), 'category'] = 8

In [ ]:
mentor_stats.category.value_counts()

category
1    542453
2     24435
7     19723
4     19606
3      5827
5      4362
6      1668
8      1558
Name: count, dtype: int64

#选择读了一个PHD但是>=1段postdoc的学者记录

In [ ]:
#">1次PhD & 1次Postdoc"
mentorship[mentorship.MenteeID.isin(mentor_stats[(mentor_stats.category==5)|(mentor_stats.category==5)|(mentor_stats.category==5)|(mentor_stats.category==5)].MenteeID)].sort_values(['MenteeID', 'StopYear'])

CID  MenteeID  MentorID  MentorshipType  \
5             8         3        16               2   
6             9         3        26               1   
574         643         3       595               1   
67           75        21        65               2   
951        1080        21       116               1   
...         ...       ...       ...             ...   
742984  1784382    839539    839540               1   
742983  1784380    839539     97941               2   
743045  1784506    839576    839578               1   
743043  1784501    839576     74609               1   
743046  1784507    839576      6455               2   

                               Institution  InstitutionMAGID  StartYear  \
5       Washington University, Saint Louis      2.044655e+08        NaN   
6                          Yale University      3.297147e+07        NaN   
574                        Yale University      3.297147e+07        NaN   
67                      Harvard University      1.362000e+08        NaN   
951                University of Cambridge      2.417490e+05        NaN   
...                                    ...               ...        ...   
742984       Weizmann Institute of Science      5.396458e+07     1995.0   
742983            UC San Diego at La Jolla               NaN     1999.0   
743045                 New York University      5.720697e+07     2013.0   
743043  California Institute of Technology      1.224118e+08     2015.0   
743046  University of California (Berkley)      2.803209e+09     2021.0   

        StopYear  Diff  
5            NaN   NaN  
6            NaN   NaN  
574          NaN   NaN  
67           NaN   NaN  
951          NaN   NaN  
...          ...   ...  
742984    2000.0   5.0  
742983    2003.0   4.0  
743045    2015.0   2.0  
743043    2021.0   6.0  
743046       NaN   NaN  

[13957 rows x 9 columns]

In [ ]:
#选择读了一个PHD但是>=1段postdoc的学者记录
mentorship_postdoc=mentorship[mentorship.MenteeID.isin(mentor_stats[(mentor_stats.category==2)|(mentor_stats.category==3)].MenteeID)]

In [ ]:
mentorship_postdoc=mentorship_postdoc.sort_values(
    by=['MenteeID', 'StopYear', 'StartYear'],
    key=lambda x: x.where(x.notna(), mentorship_postdoc['StartYear']) if x.name == 'StopYear' else x
)

把做过3段以上的博后的删掉

In [ ]:
# 方法1：直接使用groupby+size()统计
mentee_counts = mentorship_postdoc.groupby('MenteeID').size().reset_index(name='count')

# 方法2：使用value_counts()统计（结果相同）
mentee_counts = mentorship_postdoc['MenteeID'].value_counts().reset_index()
mentee_counts.columns = ['MenteeID', 'count']
#把做过3段以上的博后的删掉
mentees_to_remove = mentee_counts[mentee_counts['count'] >= 4]['MenteeID'].tolist()

#从mentorship_postdoc中过滤掉这些MenteeID
mentorship_postdoc_filtered = mentorship_postdoc[~mentorship_postdoc['MenteeID'].isin(mentees_to_remove)]

# 验证结果
print(f"原始记录数: {len(mentorship_postdoc)}")
print(f"移除的记录数: {len(mentorship_postdoc) - len(mentorship_postdoc_filtered)}")
print(f"过滤后记录数: {len(mentorship_postdoc_filtered)}")

原始记录数: 67511
移除的记录数: 4004
过滤后记录数: 63507


In [ ]:
mentorship_postdoc_filtered

CID  MenteeID  MentorID  MentorshipType  \
113068        1         1         3               1   
308         346         1       184               2   
0             2         2         3               1   
1102       1256         2        72               2   
40           48         4        52               1   
...         ...       ...       ...             ...   
743133  1784709    839653    834481               2   
743149  1784744    839668    839669               1   
743148  1784742    839668    793496               2   
743164  1784783    839674     78875               1   
743168  1784789    839674     59117               2   

                                 Institution  InstitutionMAGID  StartYear  \
113068    University of California, Berkeley        95457486.0     1998.0   
308     University of Maryland, College Park        66946132.0     2005.0   
0         University of California, Berkeley        95457486.0     2000.0   
1102                         Duke University       170897317.0     2005.0   
40                   University of Cambridge          241749.0     1999.0   
...                                      ...               ...        ...   
743133                    Charles University               NaN        NaN   
743149               University of Leeds, UK       130828816.0     1987.0   
743148              University of Manchester        28407311.0     1991.0   
743164               Northeastern University        12912129.0     1998.0   
743168                   Brandeis University         6902469.0        NaN   

        StopYear  Diff  
113068    2005.0   7.0  
308       2012.0   7.0  
0         2005.0   5.0  
1102      2011.0   6.0  
40        2002.0   3.0  
...          ...   ...  
743133       NaN   NaN  
743149    1991.0   4.0  
743148    1992.0   1.0  
743164    2003.0   5.0  
743168       NaN   NaN  

[63507 rows x 9 columns]

In [ ]:
def fill_academic_years(df):
    """
    分层填充学术阶段时间缺失值（禁用全局统计值填充）
    填充顺序：阶段内连续性填充 → 跨阶段逻辑填充 → 默认学术时长推算
    """
    # 添加填充来源标记列
    df = df.copy()
    df['StartYearSource'] = 'original'
    df['StopYearSource'] = 'original'

    # 按学员分组处理
    grouped = df.groupby('MenteeID')
    filled_dfs = []

    for mentee_id, group in grouped:
        # 分离PhD和Postdoc记录
        phd = group[group['MentorshipType'] == 1].copy()
        postdocs = group[group['MentorshipType'] == 2].copy().sort_values('StartYear')

        # ===== 阶段内连续性填充 (同类型记录间) =====
        # 1.1 填充Postdoc记录间连续性
        if len(postdocs) > 1:
            # 正向填充StartYear (用前段结束年+1)
            for i in range(1, len(postdocs)):
                prev_stop = postdocs.iloc[i-1]['StopYear']
                curr_start = postdocs.iloc[i]['StartYear']
                if pd.isna(curr_start) and not pd.isna(prev_stop):
                    postdocs.iloc[i, postdocs.columns.get_loc('StartYear')] = prev_stop
                    postdocs.iloc[i, postdocs.columns.get_loc('StartYearSource')] = 'intra_phase'

            # 反向填充StopYear (用后段开始年-1)
            for i in range(len(postdocs)-2, -1, -1):
                next_start = postdocs.iloc[i+1]['StartYear']
                curr_stop = postdocs.iloc[i]['StopYear']
                if pd.isna(curr_stop) and not pd.isna(next_start):
                    postdocs.iloc[i, postdocs.columns.get_loc('StopYear')] = next_start
                    postdocs.iloc[i, postdocs.columns.get_loc('StopYearSource')] = 'intra_phase'

        # ===== 跨阶段逻辑填充 (PhD→Postdoc) =====
        if not phd.empty and not postdocs.empty:
            # 取最近的PhD记录（按StopYear降序）
            latest_phd = phd.sort_values('StopYear', ascending=False).iloc[0]
            first_postdoc = postdocs.iloc[0]

            # 2.1 用PhD结束年填充首个Postdoc开始年
            if pd.isna(first_postdoc['StartYear']) and not pd.isna(latest_phd['StopYear']):
                postdocs.iloc[0, postdocs.columns.get_loc('StartYear')] = latest_phd['StopYear']
                postdocs.iloc[0, postdocs.columns.get_loc('StartYearSource')] = 'cross_phase'

            # 2.2 用Postdoc开始年反推PhD结束年
            if pd.isna(latest_phd['StopYear']) and not pd.isna(first_postdoc['StartYear']):
                phd.iloc[0, phd.columns.get_loc('StopYear')] = first_postdoc['StartYear']
                phd.iloc[0, phd.columns.get_loc('StopYearSource')] = 'cross_phase'

        # ===== 默认学术时长推算 =====
        # 3.1 处理PhD记录
        for i, phd_row in phd.iterrows():
            # 单缺失场景：用已知时间推算
            if pd.isna(phd_row['StopYear']) and not pd.isna(phd_row['StartYear']):
                phd.loc[i, 'StopYear'] = phd_row['StartYear'] + 5
                phd.loc[i, 'StopYearSource'] = 'duration_infer'
            elif pd.isna(phd_row['StartYear']) and not pd.isna(phd_row['StopYear']):
                phd.loc[i, 'StartYear'] = phd_row['StopYear'] - 5
                phd.loc[i, 'StartYearSource'] = 'duration_infer'

        # 3.2 处理Postdoc记录
        for i, postdoc_row in postdocs.iterrows():
            # 单缺失场景：用已知时间推算
            if pd.isna(postdoc_row['StopYear']) and not pd.isna(postdoc_row['StartYear']):
                postdocs.loc[i, 'StopYear'] = postdoc_row['StartYear'] + 3
                postdocs.loc[i, 'StopYearSource'] = 'duration_infer'
            elif pd.isna(postdoc_row['StartYear']) and not pd.isna(postdoc_row['StopYear']):
                postdocs.loc[i, 'StartYear'] = postdoc_row['StopYear'] - 3
                postdocs.loc[i, 'StartYearSource'] = 'duration_infer'

        filled_dfs.append(phd)
        filled_dfs.append(postdocs)

    # 合并结果
    result_df = pd.concat(filled_dfs)
    return result_df

# ===== 执行填充 =====
filled_df = fill_academic_years(mentorship_postdoc_filtered)

# ===== 验证与统计 =====
# 1. 检查残留缺失值
missing_after = filled_df[['StartYear', 'StopYear']].isnull().sum()
print(f"填充后缺失值统计:\nStartYear: {missing_after[0]}, StopYear: {missing_after[1]}")

# 2. 查看填充来源分布
print("\n填充来源分布:")
print(filled_df[['StartYearSource', 'StopYearSource']].apply(pd.Series.value_counts))

# 保存结果（可选）
# filled_df.to_csv('mentorship_postdoc_filled.csv', index=False)

填充后缺失值统计:
StartYear: 11272, StopYear: 11272

填充来源分布:
                StartYearSource  StopYearSource
original                  44124           52242
duration_infer            12372            9457
cross_phase                6346            1564
intra_phase                 665             244


<ipython-input-121-aa4e15de846d>:92: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"填充后缺失值统计:\nStartYear: {missing_after[0]}, StopYear: {missing_after[1]}")


In [ ]:
filled_df.describe()

CID       MenteeID       MentorID  MentorshipType  \
count  6.350700e+04   63507.000000   63507.000000    63507.000000   
mean   7.637083e+05  301093.490875  166589.935377        1.538413   
std    7.113595e+05  320760.647502  255854.387159        0.498526   
min    1.000000e+00       1.000000       1.000000        1.000000   
25%    7.047950e+04   36514.000000   11846.500000        1.000000   
50%    3.684850e+05   96334.000000   56406.000000        2.000000   
75%    1.507660e+06  693679.500000  109103.500000        2.000000   
max    1.784789e+06  839674.000000  839669.000000        2.000000   

       InstitutionMAGID     StartYear      StopYear          Diff  
count      4.867700e+04  52235.000000  52235.000000  28029.000000  
mean       1.727945e+08   1995.262334   1999.042194      3.613543  
std        3.500891e+08     25.595122     25.641169      1.782844  
min        2.417490e+05   1439.000000   1444.000000      0.000000  
25%        6.319074e+07   1990.000000   1994.000000      2.000000  
50%        1.103780e+08   2003.000000   2007.000000      4.000000  
75%        1.577252e+08   2009.000000   2013.000000      5.000000  
max        3.020458e+09   2021.000000   2025.000000      9.000000

In [ ]:
# 找出StartYear和StopYear都为空的记录
empty_years = filled_df[filled_df['StartYear'].isnull() & filled_df['StopYear'].isnull()]

# 查看结果
print(f"双字段缺失的记录数: {len(empty_years)}")
empty_years[['MenteeID', 'MentorshipType', 'StartYear', 'StopYear', 'StartYearSource', 'StopYearSource']].head()

双字段缺失的记录数: 11272


MenteeID  MentorshipType  StartYear  StopYear StartYearSource  \
83           7               1        NaN       NaN        original   
41           7               2        NaN       NaN        original   
8           14               1        NaN       NaN        original   
9           14               2        NaN       NaN        original   
1676        15               1        NaN       NaN        original   

     StopYearSource  
83         original  
41         original  
8          original  
9          original  
1676       original

In [ ]:
filled_df[filled_df.MenteeID==7]

CID  MenteeID  MentorID  MentorshipType  \
83   92         7        90               1   
41   49         7        53               2   
36   44         7         3               2   

                              Institution  InstitutionMAGID  StartYear  \
83     California Institute of Technology       122411786.0        NaN   
41  Massachusetts Institute of Technology        63966007.0        NaN   
36     University of California, Berkeley        95457486.0     2001.0   

    StopYear  Diff StartYearSource StopYearSource  
83       NaN   NaN        original       original  
41       NaN   NaN        original       original  
36    2004.0   NaN  duration_infer       original

In [ ]:
# 获取empty_years中的所有MenteeID
ids_to_remove = empty_years['MenteeID'].unique()

# 从filled_df中排除这些ID
filtered_df = filled_df[~filled_df['MenteeID'].isin(ids_to_remove)]

# 验证结果
print(f"原始记录数: {len(filled_df)}")
print(f"过滤后记录数: {len(filtered_df)}")
print(f"移除记录数: {len(filled_df) - len(filtered_df)}")

原始记录数: 63507
过滤后记录数: 50829
移除记录数: 12678


In [ ]:
filtered_df.groupby('MenteeID').count().describe()

CID      MentorID  MentorshipType   Institution  \
count  23622.000000  23622.000000    23622.000000  23622.000000   
mean       2.151765      2.151765        2.151765      2.054610   
std        0.358801      0.358801        0.358801      0.497892   
min        2.000000      2.000000        2.000000      0.000000   
25%        2.000000      2.000000        2.000000      2.000000   
50%        2.000000      2.000000        2.000000      2.000000   
75%        2.000000      2.000000        2.000000      2.000000   
max        3.000000      3.000000        3.000000      3.000000   

       InstitutionMAGID     StartYear      StopYear          Diff  \
count      23622.000000  23622.000000  23622.000000  23622.000000   
mean           1.721954      2.151765      2.151765      1.177927   
std            0.696894      0.358801      0.358801      0.855783   
min            0.000000      2.000000      2.000000      0.000000   
25%            1.000000      2.000000      2.000000      1.000000   
50%            2.000000      2.000000      2.000000      1.000000   
75%            2.000000      2.000000      2.000000      2.000000   
max            3.000000      3.000000      3.000000      3.000000   

       StartYearSource  StopYearSource  
count     23622.000000    23622.000000  
mean          2.151765        2.151765  
std           0.358801        0.358801  
min           2.000000        2.000000  
25%           2.000000        2.000000  
50%           2.000000        2.000000  
75%           2.000000        2.000000  
max           3.000000        3.000000

In [ ]:
filtered_df.to_csv('/content/drive/MyDrive/academic_tree/code/phd&postdoc.csv',index=False)

In [ ]:
filtered_df

CID  MenteeID  MentorID  MentorshipType  \
113068        1         1         3               1   
308         346         1       184               2   
0             2         2         3               1   
1102       1256         2        72               2   
40           48         4        52               1   
...         ...       ...       ...             ...   
743128  1784699    839651     26878               2   
743149  1784744    839668    839669               1   
743148  1784742    839668    793496               2   
743164  1784783    839674     78875               1   
743168  1784789    839674     59117               2   

                                          Institution  InstitutionMAGID  \
113068             University of California, Berkeley        95457486.0   
308              University of Maryland, College Park        66946132.0   
0                  University of California, Berkeley        95457486.0   
1102                                  Duke University       170897317.0   
40                            University of Cambridge          241749.0   
...                                               ...               ...   
743128  Hopkins Marine Station of Stanford University               NaN   
743149                        University of Leeds, UK       130828816.0   
743148                       University of Manchester        28407311.0   
743164                        Northeastern University        12912129.0   
743168                            Brandeis University         6902469.0   

        StartYear  StopYear  Diff StartYearSource  StopYearSource  
113068     1998.0    2005.0   7.0        original        original  
308        2005.0    2012.0   7.0        original        original  
0          2000.0    2005.0   5.0        original        original  
1102       2005.0    2011.0   6.0        original        original  
40         1999.0    2002.0   3.0        original        original  
...           ...       ...   ...             ...             ...  
743128     2017.0    2020.0   3.0        original        original  
743149     1987.0    1991.0   4.0        original        original  
743148     1991.0    1992.0   1.0        original        original  
743164     1998.0    2003.0   5.0        original        original  
743168     2003.0    2006.0   NaN     cross_phase  duration_infer  

[50829 rows x 11 columns]

#自求多福

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
phdpostdoc=pd.read_csv('/content/drive/MyDrive/academic_tree/code/phd&postdoc.csv')

In [ ]:
len(phdpostdoc['MenteeID'].unique())

23622

In [ ]:

count_stats = phdpostdoc['MenteeID'].value_counts().value_counts()

print({
    "总MenteeID数": phdpostdoc['MenteeID'].nunique(),
    "出现2次的ID数": count_stats.get(2, 0),
    "出现3次的ID数": count_stats.get(3, 0)
})

{'总MenteeID数': 23622, '出现2次的ID数': np.int64(20037), '出现3次的ID数': np.int64(3585)}


In [ ]:
MAG_paper=pd.read_csv('/content/drive/MyDrive/academic_tree/data/Scinet/MAGpaper.csv')

In [ ]:
MAG_paper.rename({'PaperID':'MAGPaperID'},axis=1,inplace=True)

In [ ]:
#authorship.csv lists all the MAG paper IDs of each researcher and has two columns: person ID (PID) and MAG paper ID
#len(authorship.MAGPaperID.unique())
#16942415
#authorship有26378069行
authorship=pd.read_csv('/content/drive/MyDrive/academic_tree/data/authorship.csv')

In [ ]:
authorship=authorship.merge(MAG_paper,on=['MAGPaperID'])

In [ ]:
authorship.shape

(22195049, 29)

In [ ]:
authorship.columns

Index(['PID', 'MAGPaperID', 'DOI', 'DocType', 'Year', 'Date', 'JournalID',
       'ConferenceSeriesID', 'Citation_Count', 'C10', 'Reference_Count', 'C5',
       'Team_Size', 'Institution_Count', 'Disruption', 'Atyp_10pct_Z',
       'Atyp_Pairs', 'Atyp_Median_Z', 'SB_B', 'SB_T', 'Patent_Count',
       'Newsfeed_Count', 'Tweet_Count', 'NCT_Count', 'NIH_Count', 'NSF_Count',
       'WSB_mu', 'WSB_sigma', 'WSB_Cinf'],
      dtype='object')

In [ ]:
authorship=authorship.drop(['Newsfeed_Count', 'Tweet_Count', 'NCT_Count', 'DOI', 'Date', 'WSB_mu', 'WSB_sigma', 'WSB_Cinf','DocType', 'JournalID',
       'ConferenceSeriesID'],axis=1)

In [ ]:
authorship

PID  MAGPaperID    Year  Citation_Count    C10  Reference_Count  \
0              1  1519578210  2015.0           383.0    NaN             57.0   
1              1  1972926514  2012.0            38.0    NaN              5.0   
2              1  1981764347  2009.0           111.0   83.0             47.0   
3              1  1982956265  2008.0           128.0  102.0             53.0   
4              1  1990434553  2008.0             6.0    5.0              0.0   
...          ...         ...     ...             ...    ...              ...   
22195044  839680  2476168493  2008.0             1.0    1.0            171.0   
22195045  839680  2494182827  1995.0             1.0    1.0             14.0   
22195046  839680  2899949966  2019.0             3.0    NaN             35.0   
22195047  839680  2965878599  1996.0             0.0    0.0              0.0   
22195048  839680  2971411371  2020.0             1.0    NaN             24.0   

             C5  Team_Size  Institution_Count  Disruption  Atyp_10pct_Z  \
0         304.0        3.0                2.0   -0.016115     -1.587336   
1           6.0        2.0                2.0   -0.013774    -27.872626   
2          26.0        4.0                1.0   -0.000531     -2.657346   
3          39.0        4.0                1.0   -0.004891     -3.622292   
4           3.0        4.0                1.0         NaN           NaN   
...         ...        ...                ...         ...           ...   
22195044    1.0        5.0                NaN   -0.000058           NaN   
22195045    1.0        5.0                NaN   -0.000869           NaN   
22195046    NaN        5.0                2.0    0.000049     -0.844119   
22195047    0.0        3.0                NaN         NaN           NaN   
22195048    NaN        5.0                3.0   -0.000060     -6.124032   

          Atyp_Pairs  Atyp_Median_Z       SB_B  SB_T  Patent_Count  NIH_Count  \
0              334.0     162.543790  -1.253290   2.0             0          4   
1                3.0     -10.887451  10.500000   4.0             1          0   
2              218.0      98.822970   2.250000   9.0             2          4   
3              221.0     206.024601   3.510921   8.0             0          1   
4                NaN            NaN   1.000000   1.0             0          0   
...              ...            ...        ...   ...           ...        ...   
22195044         NaN            NaN        NaN   NaN             0          0   
22195045         NaN            NaN        NaN   NaN             0          0   
22195046       232.0      29.897775   0.000000   0.0             0          0   
22195047         NaN            NaN        NaN   NaN             0          0   
22195048       182.0      14.472001   0.000000   0.0             0          0   

          NSF_Count  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  
...             ...  
22195044          0  
22195045          0  
22195046          0  
22195047          0  
22195048          0  

[22195049 rows x 18 columns]

In [ ]:
phdpostdoc['Diff']=phdpostdoc['StopYear']-phdpostdoc['StartYear']

In [ ]:
phdpostdoc['StartYear'] = pd.to_numeric(phdpostdoc['StartYear'], errors='coerce')
phdpostdoc['StopYear'] = pd.to_numeric(phdpostdoc['StopYear'], errors='coerce')
authorship['Year'] = pd.to_numeric(authorship['Year'], errors='coerce')

# 合并两个表（保留所有phdpostdoc记录）
merged = pd.merge(
    phdpostdoc,
    authorship,
    left_on='MenteeID',
    right_on='PID',
)

In [ ]:
# 筛选在学术阶段内发表的论文
valid_papers = merged[
    (merged['Year'] >= merged['StartYear']) &
    (merged['Year'] < merged['StopYear'])
]
paper_lists = valid_papers.groupby('CID')['MAGPaperID'].apply(list).reset_index()
paper_lists.columns = ['CID', 'PublishedPapers']

# 合并回原表
phdpostdoc = pd.merge(phdpostdoc, paper_lists, on='CID', how='left')

# 空列表填充为NaN（可选）
phdpostdoc['PublishedPapers'] = phdpostdoc['PublishedPapers'].apply(lambda x: x if isinstance(x, list) else [])

# 结果展示

In [ ]:
phdpostdoc

CID  MenteeID  MentorID  MentorshipType  \
0            1         1         3               1   
1          346         1       184               2   
2            2         2         3               1   
3         1256         2        72               2   
4           48         4        52               1   
...        ...       ...       ...             ...   
50824  1784699    839651     26878               2   
50825  1784744    839668    839669               1   
50826  1784742    839668    793496               2   
50827  1784783    839674     78875               1   
50828  1784789    839674     59117               2   

                                         Institution  InstitutionMAGID  \
0                 University of California, Berkeley        95457486.0   
1               University of Maryland, College Park        66946132.0   
2                 University of California, Berkeley        95457486.0   
3                                    Duke University       170897317.0   
4                            University of Cambridge          241749.0   
...                                              ...               ...   
50824  Hopkins Marine Station of Stanford University               NaN   
50825                        University of Leeds, UK       130828816.0   
50826                       University of Manchester        28407311.0   
50827                        Northeastern University        12912129.0   
50828                            Brandeis University         6902469.0   

       StartYear  StopYear  Diff StartYearSource  StopYearSource  \
0         1998.0    2005.0   7.0        original        original   
1         2005.0    2012.0   7.0        original        original   
2         2000.0    2005.0   5.0        original        original   
3         2005.0    2011.0   6.0        original        original   
4         1999.0    2002.0   3.0        original        original   
...          ...       ...   ...             ...             ...   
50824     2017.0    2020.0   3.0        original        original   
50825     1987.0    1991.0   4.0        original        original   
50826     1991.0    1992.0   1.0        original        original   
50827     1998.0    2003.0   5.0        original        original   
50828     2003.0    2006.0   NaN     cross_phase  duration_infer   

                                         PublishedPapers  
0       [2061209400, 2076398027, 2170556807, 2171525519]  
1      [1981764347, 1982956265, 1990434553, 200157118...  
2                                           [2165789639]  
3      [1538084273, 1982956265, 1991892677, 200361432...  
4                                                     []  
...                                                  ...  
50824                           [2763840109, 2969897880]  
50825                                                 []  
50826                                                 []  
50827               [1547812862, 1582622856, 1993122780]  
50828               [1914307507, 2077050825, 2913673587]  

[50829 rows x 12 columns]

In [ ]:
# 获取差集（在phdpostdoc但不在valid_rows中的MenteeID）
diff_ids = set(phdpostdoc.MenteeID.unique()) - set(valid_rows.MenteeID.unique())

In [ ]:
authorship[authorship.PID==84149]

PID  MAGPaperID    Year  Citation_Count  C10  Reference_Count   C5  \
4621773  84149  2063916243  1971.0             6.0  4.0              3.0  2.0   

         Team_Size  Institution_Count  Disruption  Atyp_10pct_Z  Atyp_Pairs  \
4621773        4.0                1.0         0.0     -0.895191         3.0   

         Atyp_Median_Z  SB_B  SB_T  Patent_Count  NIH_Count  NSF_Count  
4621773      -0.099757   3.0   3.0             4          0          0

In [ ]:
phdpostdoc[phdpostdoc.MenteeID==84149]

CID  MenteeID  MentorID  MentorshipType         Institution  \
21891  107679     84149     51251               1   Leiden University   
21892  107680     84149     20839               2  Cornell University   

       InstitutionMAGID  StartYear  StopYear  Diff StartYearSource  \
21891       121797337.0     1950.0    1955.0   NaN  duration_infer   
21892       205783295.0     1955.0    1957.0   2.0        original   

      StopYearSource PublishedPapers  
21891       original              []  
21892       original              []

In [ ]:
# 或更简洁的写法（确保列中都是列表类型）
valid_rows=phdpostdoc[phdpostdoc['PublishedPapers'].str.len() > 0]
valid_rows.groupby(['MenteeID','MentorshipType'])['PublishedPapers'].apply(list).reset_index()
#等价于

MenteeID  MentorshipType  \
0             1               1   
1             1               2   
2             2               1   
3             2               2   
4             4               2   
...         ...             ...   
34211    839647               2   
34212    839651               1   
34213    839651               2   
34214    839674               1   
34215    839674               2   

                                         PublishedPapers  
0      [[2061209400, 2076398027, 2170556807, 21715255...  
1      [[1981764347, 1982956265, 1990434553, 20015711...  
2                                         [[2165789639]]  
3      [[1538084273, 1982956265, 1991892677, 20036143...  
4                                         [[2121693165]]  
...                                                  ...  
34211                                     [[2093178666]]  
34212                         [[1879522373, 2150987923]]  
34213                         [[2763840109, 2969897880]]  
34214             [[1547812862, 1582622856, 1993122780]]  
34215             [[1914307507, 2077050825, 2913673587]]  

[34216 rows x 3 columns]

In [ ]:
valid_rows.head(10)

CID  MenteeID  MentorID  MentorshipType  \
0       1         1         3               1   
1     346         1       184               2   
2       2         2         3               1   
3    1256         2        72               2   
5       3         4         3               2   
7      25         5         3               1   
8   38845         5       312               2   
9      19         8         3               1   
10   4652         8        45               2   
11     10        11        10               1   

                             Institution  InstitutionMAGID  StartYear  \
0     University of California, Berkeley        95457486.0     1998.0   
1   University of Maryland, College Park        66946132.0     2005.0   
2     University of California, Berkeley        95457486.0     2000.0   
3                        Duke University       170897317.0     2005.0   
5     University of California, Berkeley        95457486.0     2003.0   
7     University of California, Berkeley        95457486.0     2003.0   
8                    Stanford University        97018004.0     2009.0   
9     University of California, Berkeley        95457486.0     2001.0   
10   National Institute of Mental Health               NaN     2006.0   
11              Johns Hopkins University       145311948.0     2001.0   

    StopYear  Diff StartYearSource StopYearSource  \
0     2005.0   7.0        original       original   
1     2012.0   7.0        original       original   
2     2005.0   5.0        original       original   
3     2011.0   6.0        original       original   
5     2006.0   3.0        original       original   
7     2009.0   6.0        original       original   
8     2013.0   4.0        original       original   
9     2006.0   5.0        original       original   
10    2012.0   6.0        original       original   
11    2006.0   NaN  duration_infer       original   

                                      PublishedPapers  
0    [2061209400, 2076398027, 2170556807, 2171525519]  
1   [1981764347, 1982956265, 1990434553, 200157118...  
2                                        [2165789639]  
3   [1538084273, 1982956265, 1991892677, 200361432...  
5                                        [2121693165]  
7    [1967303766, 2007226897, 2008402640, 2114000517]  
8   [1528508863, 2016196840, 2063887657, 206395148...  
9                                        [2061209400]  
10                           [1967303766, 2008402640]  
11                                       [2171105099]

我需要在两个阶段都有发表的学者

代表了每个学者MenteeID，在不同阶段MentorshipType（=1or=2)的发表MAGPaperID，有的因为MAGPaperID的内容是一个空列表被去掉了，因此该MenteeID只有一个记录了（MentorshipType=1或者2），需要删掉这些人

In [ ]:
grouped = valid_papers.groupby(['MenteeID','MentorshipType'])['MAGPaperID'].apply(list).reset_index()
# 统计每个MenteeID拥有的MentorshipType种类数
type_counts = grouped.groupby('MenteeID')['MentorshipType'].nunique()
# 单阶段学员ID列表
single_type_mentees = type_counts[type_counts == 1].index.tolist()
dual_type_mentees = type_counts[type_counts == 2].index.tolist()

In [ ]:
# 筛选双阶段学员
dual_data = grouped[grouped['MenteeID'].isin(dual_type_mentees)]

# 转换为宽表格式
result_df = (
    dual_data.pivot(index='MenteeID', columns='MentorshipType', values='MAGPaperID')
    .reset_index()
    .rename(columns={1: 'PhD_Papers', 2: 'Postdoc_Papers'})
)


In [ ]:
result_df

MentorshipType  MenteeID                                         PhD_Papers  \
0                      1   [2061209400, 2076398027, 2170556807, 2171525519]   
1                      2                                       [2165789639]   
2                      5   [1967303766, 2007226897, 2008402640, 2114000517]   
3                      8                                       [2061209400]   
4                     11                                       [2171105099]   
...                  ...                                                ...   
14691             839561  [1566023115, 1984247103, 2031740525, 203450916...   
14692             839562  [1586856815, 2022309091, 2031585901, 206099992...   
14693             839563  [1670888144, 1981058958, 1996398789, 204781795...   
14694             839651                           [1879522373, 2150987923]   
14695             839674               [1547812862, 1582622856, 1993122780]   

MentorshipType                                     Postdoc_Papers  
0               [1981764347, 1982956265, 1990434553, 200157118...  
1               [1538084273, 1982956265, 1991892677, 200361432...  
2               [1528508863, 2016196840, 2063887657, 206395148...  
3                                        [1967303766, 2008402640]  
4                                        [1985106404, 2015449083]  
...                                                           ...  
14691           [2337042362, 2549776610, 2602595188, 261569665...  
14692           [1567886374, 1966079519, 1968692389, 201454541...  
14693           [157597599, 1610453553, 1632151204, 1640410376...  
14694                                    [2763840109, 2969897880]  
14695                        [1914307507, 2077050825, 2913673587]  

[14696 rows x 3 columns]

去掉那种PhD_Papers或者Postdoc_Papers的list里面只有一个元素的行

In [ ]:
result_df = result_df[
    (result_df['PhD_Papers'].apply(len) > 1) &
    (result_df['Postdoc_Papers'].apply(len) > 1)
]

#去掉那种

In [ ]:
result_df

MentorshipType  MenteeID                                         PhD_Papers  \
0                      1   [2061209400, 2076398027, 2170556807, 2171525519]   
2                      5   [1967303766, 2007226897, 2008402640, 2114000517]   
5                     13               [1808792747, 2056851088, 2072308568]   
6                     17    [132109560, 2058670155, 2100035604, 2171525519]   
7                     20  [2016961720, 2068335026, 2095399752, 211983173...   
...                  ...                                                ...   
14691             839561  [1566023115, 1984247103, 2031740525, 203450916...   
14692             839562  [1586856815, 2022309091, 2031585901, 206099992...   
14693             839563  [1670888144, 1981058958, 1996398789, 204781795...   
14694             839651                           [1879522373, 2150987923]   
14695             839674               [1547812862, 1582622856, 1993122780]   

MentorshipType                                     Postdoc_Papers  
0               [1981764347, 1982956265, 1990434553, 200157118...  
2               [1528508863, 2016196840, 2063887657, 206395148...  
5                                        [1777366511, 2005108691]  
6                [1589049074, 2102662817, 2114219449, 2170556807]  
7                            [2075772363, 2165450323, 2439980839]  
...                                                           ...  
14691           [2337042362, 2549776610, 2602595188, 261569665...  
14692           [1567886374, 1966079519, 1968692389, 201454541...  
14693           [157597599, 1610453553, 1632151204, 1640410376...  
14694                                    [2763840109, 2969897880]  
14695                        [1914307507, 2077050825, 2913673587]  

[10942 rows x 3 columns]

In [ ]:
researcher_neighbor_specter=pd.read_csv('/content/drive/MyDrive/academic_tree/data/researcher_neighbor_specter.csv.gz')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
researcher_neighbor_specter['SpecterDistance_scaled'] = scaler.fit_transform(researcher_neighbor_specter[['SpecterDistance']])
pid_means = researcher_neighbor_specter.groupby('PID')['SpecterDistance_scaled'].mean().reset_index()

In [ ]:
pid_means

PID  SpecterDistance_scaled
0            1                0.038790
1            2                0.052517
2            3                0.043112
3            4                0.124004
4            5                0.044751
...        ...                     ...
494450  839676                0.070336
494451  839677                0.097104
494452  839678                0.082714
494453  839679                0.270334
494454  839680                0.107194

[494455 rows x 2 columns]

In [ ]:
from scipy import sparse
data = np.load('/content/drive/MyDrive/academic_tree/data/tfidf/paper_tfidf.npz')
tfidf_csr = sparse.csr_matrix(
    (data['data'], data['indices'], data['indptr']),
    shape=data['shape']
)

paper_tfidf_MAGPaperID = pd.read_csv(
    '/content/drive/MyDrive/academic_tree/data/tfidf/paper_tfidf_MAGPaperID.txt',
    compression='gzip',
    header=None,  # 无列名
    names=['MAGPaperID']  # 自定义列名
)

In [ ]:
tfidf_csr[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3 stored elements and shape (1, 2275293)>

In [ ]:
paper_id_mapping = paper_tfidf_MAGPaperID.to_dict()  # {索引: 值}
reversed_dict = {v: k for k, v in paper_id_mapping['MAGPaperID'].items()}

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_paper_indices(paper_ids):
    return [reversed_dict[pid] for pid in paper_ids]

def calc_phase_similarity(paper_list):
    """计算一个阶段论文集合的余弦相似度均值"""
    if len(paper_list) < 2:
        return np.nan  # 不足2篇无法计算

    # 获取所有论文的TF-IDF向量
    indices = get_paper_indices(paper_list)
    vectors = tfidf_csr[indices]

    # 计算全连接相似度矩阵
    sim_matrix = cosine_similarity(vectors)

    # 返回非对角元素的均值（排除自相似）
    return np.tril(sim_matrix, k=-1).sum() / (len(paper_list)*(len(paper_list)-1)/2)


In [ ]:
from tqdm import tqdm
result_df['PhD_Similarity'] = [calc_phase_similarity(papers) for papers in tqdm(result_df['PhD_Papers'])]

100%|██████████| 10942/10942 [00:34<00:00, 314.57it/s]
<ipython-input-145-f1ac07a47f13>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['PhD_Similarity'] = [calc_phase_similarity(papers) for papers in tqdm(result_df['PhD_Papers'])]


In [ ]:
# 计算每个学员两个阶段的相似度
result_df['PhD_Similarity'] = result_df['PhD_Papers'].apply(calc_phase_similarity)
result_df['Postdoc_Similarity'] = result_df['Postdoc_Papers'].apply(calc_phase_similarity)

# 处理NaN值（可选）
#result_df.fillna({'PhD_Similarity': -1, 'Postdoc_Similarity': -1}, inplace=True)

<ipython-input-147-f763773521d6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['PhD_Similarity'] = result_df['PhD_Papers'].apply(calc_phase_similarity)
<ipython-input-147-f763773521d6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Postdoc_Similarity'] = result_df['Postdoc_Papers'].apply(calc_phase_similarity)


In [ ]:
result_df

MentorshipType  MenteeID                                         PhD_Papers  \
0                      1   [2061209400, 2076398027, 2170556807, 2171525519]   
2                      5   [1967303766, 2007226897, 2008402640, 2114000517]   
5                     13               [1808792747, 2056851088, 2072308568]   
6                     17    [132109560, 2058670155, 2100035604, 2171525519]   
7                     20  [2016961720, 2068335026, 2095399752, 211983173...   
...                  ...                                                ...   
14691             839561  [1566023115, 1984247103, 2031740525, 203450916...   
14692             839562  [1586856815, 2022309091, 2031585901, 206099992...   
14693             839563  [1670888144, 1981058958, 1996398789, 204781795...   
14694             839651                           [1879522373, 2150987923]   
14695             839674               [1547812862, 1582622856, 1993122780]   

MentorshipType                                     Postdoc_Papers  \
0               [1981764347, 1982956265, 1990434553, 200157118...   
2               [1528508863, 2016196840, 2063887657, 206395148...   
5                                        [1777366511, 2005108691]   
6                [1589049074, 2102662817, 2114219449, 2170556807]   
7                            [2075772363, 2165450323, 2439980839]   
...                                                           ...   
14691           [2337042362, 2549776610, 2602595188, 261569665...   
14692           [1567886374, 1966079519, 1968692389, 201454541...   
14693           [157597599, 1610453553, 1632151204, 1640410376...   
14694                                    [2763840109, 2969897880]   
14695                        [1914307507, 2077050825, 2913673587]   

MentorshipType  PhD_Similarity  Postdoc_Similarity  
0                     0.255856            0.131430  
2                     0.098599            0.151952  
5                     0.080222            0.412325  
6                     0.127348            0.130017  
7                     0.220879            0.292287  
...                        ...                 ...  
14691                 0.086157            0.081637  
14692                 0.070873            0.151306  
14693                 0.152863            0.033027  
14694                 0.574214            0.072660  
14695                 0.103617            0.049717  

[10942 rows x 5 columns]

In [ ]:
def calc_cross_similarity(phd_ids, postdoc_ids):
    # 获取有效论文的索引
    phd_indices = [reversed_dict[pid] for pid in phd_ids]
    postdoc_indices = [reversed_dict[pid] for pid in postdoc_ids]

    if not phd_indices or not postdoc_indices:
        return np.nan

    # 获取TF-IDF向量
    phd_vectors = tfidf_csr[phd_indices]
    postdoc_vectors = tfidf_csr[postdoc_indices]

    # 计算跨列表相似度矩阵
    sim_matrix = cosine_similarity(phd_vectors, postdoc_vectors)

    # 返回所有组合的均值
    return sim_matrix.mean()

In [ ]:
result_df['Cross_Similarity'] = result_df.apply(
    lambda row: calc_cross_similarity(row['PhD_Papers'], row['Postdoc_Papers']),
    axis=1
)

<ipython-input-152-5bd246d65bce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Cross_Similarity'] = result_df.apply(


In [ ]:
result_df[['PhD_Similarity',
       'Postdoc_Similarity', 'Cross_Similarity']].corr()

MentorshipType      PhD_Similarity  Postdoc_Similarity  Cross_Similarity
MentorshipType                                                          
PhD_Similarity            1.000000            0.154498          0.422772
Postdoc_Similarity        0.154498            1.000000          0.462469
Cross_Similarity          0.422772            0.462469          1.000000

In [ ]:
result_df

MentorshipType  MenteeID                                         PhD_Papers  \
0                      1   [2061209400, 2076398027, 2170556807, 2171525519]   
2                      5   [1967303766, 2007226897, 2008402640, 2114000517]   
5                     13               [1808792747, 2056851088, 2072308568]   
6                     17    [132109560, 2058670155, 2100035604, 2171525519]   
7                     20  [2016961720, 2068335026, 2095399752, 211983173...   
...                  ...                                                ...   
14691             839561  [1566023115, 1984247103, 2031740525, 203450916...   
14692             839562  [1586856815, 2022309091, 2031585901, 206099992...   
14693             839563  [1670888144, 1981058958, 1996398789, 204781795...   
14694             839651                           [1879522373, 2150987923]   
14695             839674               [1547812862, 1582622856, 1993122780]   

MentorshipType                                     Postdoc_Papers  \
0               [1981764347, 1982956265, 1990434553, 200157118...   
2               [1528508863, 2016196840, 2063887657, 206395148...   
5                                        [1777366511, 2005108691]   
6                [1589049074, 2102662817, 2114219449, 2170556807]   
7                            [2075772363, 2165450323, 2439980839]   
...                                                           ...   
14691           [2337042362, 2549776610, 2602595188, 261569665...   
14692           [1567886374, 1966079519, 1968692389, 201454541...   
14693           [157597599, 1610453553, 1632151204, 1640410376...   
14694                                    [2763840109, 2969897880]   
14695                        [1914307507, 2077050825, 2913673587]   

MentorshipType  PhD_Similarity  Postdoc_Similarity  Cross_Similarity  
0                     0.255856            0.131430          0.122978  
2                     0.098599            0.151952          0.128381  
5                     0.080222            0.412325          0.130528  
6                     0.127348            0.130017          0.156601  
7                     0.220879            0.292287          0.208777  
...                        ...                 ...               ...  
14691                 0.086157            0.081637          0.050875  
14692                 0.070873            0.151306          0.062724  
14693                 0.152863            0.033027          0.038650  
14694                 0.574214            0.072660          0.185077  
14695                 0.103617            0.049717          0.056865  

[10942 rows x 6 columns]

In [ ]:
authorship['C10'] = authorship['C10'].fillna(authorship['Citation_Count'])
authorship['C5'] = authorship['C5'].fillna(authorship['Citation_Count'])

In [ ]:
authorship

PID  MAGPaperID    Year  Citation_Count    C10  Reference_Count  \
0              1  1519578210  2015.0           383.0  383.0             57.0   
1              1  1972926514  2012.0            38.0   38.0              5.0   
2              1  1981764347  2009.0           111.0   83.0             47.0   
3              1  1982956265  2008.0           128.0  102.0             53.0   
4              1  1990434553  2008.0             6.0    5.0              0.0   
...          ...         ...     ...             ...    ...              ...   
22195044  839680  2476168493  2008.0             1.0    1.0            171.0   
22195045  839680  2494182827  1995.0             1.0    1.0             14.0   
22195046  839680  2899949966  2019.0             3.0    3.0             35.0   
22195047  839680  2965878599  1996.0             0.0    0.0              0.0   
22195048  839680  2971411371  2020.0             1.0    1.0             24.0   

             C5  Team_Size  Institution_Count  Disruption  Atyp_10pct_Z  \
0         304.0        3.0                2.0   -0.016115     -1.587336   
1           6.0        2.0                2.0   -0.013774    -27.872626   
2          26.0        4.0                1.0   -0.000531     -2.657346   
3          39.0        4.0                1.0   -0.004891     -3.622292   
4           3.0        4.0                1.0         NaN           NaN   
...         ...        ...                ...         ...           ...   
22195044    1.0        5.0                NaN   -0.000058           NaN   
22195045    1.0        5.0                NaN   -0.000869           NaN   
22195046    3.0        5.0                2.0    0.000049     -0.844119   
22195047    0.0        3.0                NaN         NaN           NaN   
22195048    1.0        5.0                3.0   -0.000060     -6.124032   

          Atyp_Pairs  Atyp_Median_Z       SB_B  SB_T  Patent_Count  NIH_Count  \
0              334.0     162.543790  -1.253290   2.0             0          4   
1                3.0     -10.887451  10.500000   4.0             1          0   
2              218.0      98.822970   2.250000   9.0             2          4   
3              221.0     206.024601   3.510921   8.0             0          1   
4                NaN            NaN   1.000000   1.0             0          0   
...              ...            ...        ...   ...           ...        ...   
22195044         NaN            NaN        NaN   NaN             0          0   
22195045         NaN            NaN        NaN   NaN             0          0   
22195046       232.0      29.897775   0.000000   0.0             0          0   
22195047         NaN            NaN        NaN   NaN             0          0   
22195048       182.0      14.472001   0.000000   0.0             0          0   

          NSF_Count  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  
...             ...  
22195044          0  
22195045          0  
22195046          0  
22195047          0  
22195048          0  

[22195049 rows x 18 columns]

In [ ]:
# 步骤1：展开result_df中的Postdoc_Papers列表
expanded_df = result_df.explode('Postdoc_Papers')

In [ ]:
# 步骤2：与authorship表合并
merged_df = pd.merge(
    expanded_df[['MenteeID', 'Postdoc_Papers','PhD_Similarity',
       'Postdoc_Similarity', 'Cross_Similarity']],
    authorship,
    left_on='Postdoc_Papers',
    right_on='MAGPaperID',
    how='left'
)

# 步骤3：计算各项指标的平均值
metrics = ['Citation_Count', 'C10', 'C5', 'Team_Size',
           'Institution_Count', 'Disruption', 'NIH_Count', 'NSF_Count','PhD_Similarity',
       'Postdoc_Similarity', 'Cross_Similarity']

result_metrics = merged_df.groupby('MenteeID')[metrics].mean().reset_index()

In [ ]:
result_metrics.corr()

MenteeID  Citation_Count       C10        C5  Team_Size  \
MenteeID            1.000000       -0.071450 -0.071171 -0.019950   0.036115   
Citation_Count     -0.071450        1.000000  0.611051  0.523931   0.039991   
C10                -0.071171        0.611051  1.000000  0.947134   0.088303   
C5                 -0.019950        0.523931  0.947134  1.000000   0.122540   
Team_Size           0.036115        0.039991  0.088303  0.122540   1.000000   
Institution_Count   0.043570        0.090031  0.190963  0.264833   0.816411   
Disruption          0.037938        0.004745  0.016606  0.021748   0.013240   
NIH_Count          -0.090428        0.130828  0.275390  0.315516   0.111252   
NSF_Count           0.058151        0.024879  0.082800  0.097010   0.310143   
PhD_Similarity      0.006635       -0.003488 -0.003805 -0.004477   0.023874   
Postdoc_Similarity -0.044709       -0.007071 -0.035111 -0.041020   0.003661   
Cross_Similarity   -0.050181       -0.009144 -0.027505 -0.034150   0.008014   

                    Institution_Count  Disruption  NIH_Count  NSF_Count  \
MenteeID                     0.043570    0.037938  -0.090428   0.058151   
Citation_Count               0.090031    0.004745   0.130828   0.024879   
C10                          0.190963    0.016606   0.275390   0.082800   
C5                           0.264833    0.021748   0.315516   0.097010   
Team_Size                    0.816411    0.013240   0.111252   0.310143   
Institution_Count            1.000000    0.018703   0.324290   0.263728   
Disruption                   0.018703    1.000000   0.044106   0.040311   
NIH_Count                    0.324290    0.044106   1.000000  -0.048355   
NSF_Count                    0.263728    0.040311  -0.048355   1.000000   
PhD_Similarity               0.017246    0.020756   0.033481  -0.045254   
Postdoc_Similarity          -0.016530    0.004822  -0.000821  -0.044244   
Cross_Similarity            -0.007069    0.031384   0.020000  -0.039323   

                    PhD_Similarity  Postdoc_Similarity  Cross_Similarity  
MenteeID                  0.006635           -0.044709         -0.050181  
Citation_Count           -0.003488           -0.007071         -0.009144  
C10                      -0.003805           -0.035111         -0.027505  
C5                       -0.004477           -0.041020         -0.034150  
Team_Size                 0.023874            0.003661          0.008014  
Institution_Count         0.017246           -0.016530         -0.007069  
Disruption                0.020756            0.004822          0.031384  
NIH_Count                 0.033481           -0.000821          0.020000  
NSF_Count                -0.045254           -0.044244         -0.039323  
PhD_Similarity            1.000000            0.154498          0.422772  
Postdoc_Similarity        0.154498            1.000000          0.462469  
Cross_Similarity          0.422772            0.462469          1.000000

In [ ]:
result_metrics.to_csv('data.csv',index=False)